## facial point 정보를 수집하여 csv로 저장

* image-url로 부터 facial point 수집하는 api

In [2]:
import urllib.parse
import urllib.request
import json

## face detect api call 
def face_detect(imgurl):
    url = 'http://apius.faceplusplus.com/v2/detection/detect'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'attribute' : 'none',
              'url' : imgurl,
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}

    data = urllib.parse.urlencode(values)
    data = data.encode(u'utf-8')
    req = urllib.request.Request(url, data, headers)
    response = urllib.request.urlopen(req)
    response_data = response.read()
    response.close()
    return json.loads(response_data.decode('utf-8'))

## face landmark api call by face_id
def face_landmark(faceid):
    url = 'http://apius.faceplusplus.com/v2/detection/landmark'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'face_id' : faceid,
              'type' : '83p',
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}
    data = urllib.parse.urlencode(values)
    data = data.encode(u'utf-8')

    req = urllib.request.Request(url, data, headers)

    response = urllib.request.urlopen(req)
    response_data = response.read()
    response.close()
    return json.loads(response_data.decode('utf-8'))

def  extract_face_landmark( imgurl, face_index=0 ) :
    fd = face_detect(imgurl)
    face_id = fd['face'][face_index]['face_id']
    return face_landmark(face_id)
    

In [3]:
extract_face_landmark('https://fbcdn-sphotos-g-a.akamaihd.net/hphotos-ak-xta1/v/t1.0-9/12122398_906503296086905_9053689686321602818_n.jpg?oh=80fcaf06feebc607119139261daa82ee&oe=5696C59E&__gda__=1452163095_656cdb05f7f03f0920879b17b79041be' )

{'result': [{'face_id': '34ca78344b8933b9d0392991723311da',
   'landmark': {'contour_chin': {'x': 46.870333, 'y': 39.516444},
    'contour_left1': {'x': 39.563, 'y': 24.433778},
    'contour_left2': {'x': 39.6425, 'y': 26.722},
    'contour_left3': {'x': 39.836667, 'y': 28.721333},
    'contour_left4': {'x': 40.219, 'y': 30.990889},
    'contour_left5': {'x': 40.659333, 'y': 32.897778},
    'contour_left6': {'x': 41.447333, 'y': 34.854222},
    'contour_left7': {'x': 42.452333, 'y': 36.781556},
    'contour_left8': {'x': 43.545833, 'y': 38.136667},
    'contour_left9': {'x': 45.202167, 'y': 39.372444},
    'contour_right1': {'x': 54.509167, 'y': 22.996222},
    'contour_right2': {'x': 54.733833, 'y': 25.442667},
    'contour_right3': {'x': 54.652167, 'y': 27.907778},
    'contour_right4': {'x': 54.464167, 'y': 30.252222},
    'contour_right5': {'x': 53.924167, 'y': 32.507333},
    'contour_right6': {'x': 53.326167, 'y': 34.674667},
    'contour_right7': {'x': 52.0905, 'y': 36.589111},


In [37]:
import json
import numpy as np
from scipy.spatial import distance

def pos_distance(l,r):
#    print(l,r)
    return round(distance.euclidean( [l['x'], l['y']], [r['x'], r['y']] ),5)

def pos_center(l,r):
    return {'x':(l['x']+r['x'])/2, 'y':(l['y']+r['y'])/2}

def extract_face_feature(facial_info):
    cal_items = {'face_width'    :['contour_left9', 'contour_right9']
                 ,'face_height'  :['eyebrow_center','contour_chin']
                 ,'mouth_width'  :['mouth_left_corner','mouth_right_corner']  
                 ,'nose_width'   :['nose_left','nose_right']  
                 ,'eye_width'    :['left_eye_left_corner','right_eye_right_corner']  
                 ,'eye_to_eye'   :['left_eye_center','right_eye_center']  
                 ,'eyebrow_width':['left_eyebrow_upper_middle','left_eyebrow_lower_middle']  
                 ,'lip_to_chin'  :['mouth_lower_lip_bottom','contour_chin']  
                 ,'lip_to_nose'  :['nose_tip','mouth_upper_lip_top']}
    
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    ## add center position
    facial_pos['eyebrow_center'] = pos_center( facial_pos['left_eyebrow_upper_middle'], facial_pos['right_eyebrow_upper_middle']) 
    
    ## calculate key point
    for (k,v) in cal_items.items(): ## iteritems() for python2.7
        keypoint[k] = pos_distance(facial_pos[v[0]], facial_pos[v[1]])
        
    return keypoint

def normalize_dict(data, max_value):
    for k in data.keys():
        data[k] = round(data[k]/max_value,5)
    return data

def get_feature_vector( keypoint ):
    ff = extract_face_feature(keypoint)
    ff = normalize_dict(ff, ff['face_height'])
    return ff

In [49]:
import json
def extract_face_feature_raw(facial_info):
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    dx = facial_pos['contour_left1']['x']
    dy = facial_pos['contour_left1']['y']
    ## calculate key point
    for (k,v) in facial_pos.items(): ## iteritems() for python2.7
        keypoint[k+'_x'] = v['x'] - dx
        keypoint[k+'_y'] = v['y'] - dy
        
    return keypoint

#keypoint = extract_face_landmark('https://fbcdn-sphotos-g-a.akamaihd.net/hphotos-ak-xta1/v/t1.0-9/12122398_906503296086905_9053689686321602818_n.jpg?oh=80fcaf06feebc607119139261daa82ee&oe=5696C59E&__gda__=1452163095_656cdb05f7f03f0920879b17b79041be')
extract_face_feature_raw(keypoint)

{'contour_chin_x': 31.888402999999997,
 'contour_chin_y': 48.06616,
 'contour_left1_x': 0.0,
 'contour_left1_y': 0.0,
 'contour_left2_x': -0.39410700000000176,
 'contour_left2_y': 7.196577999999995,
 'contour_left3_x': 0.1577939999999991,
 'contour_left3_y': 14.544486999999997,
 'contour_left4_x': 0.689352999999997,
 'contour_left4_y': 21.93365,
 'contour_left5_x': 2.1309879999999986,
 'contour_left5_y': 28.323954,
 'contour_left6_x': 5.565398999999999,
 'contour_left6_y': 34.78821299999999,
 'contour_left7_x': 10.187641999999997,
 'contour_left7_y': 40.15,
 'contour_left8_x': 16.360645999999996,
 'contour_left8_y': 44.709506,
 'contour_left9_x': 23.885931,
 'contour_left9_y': 47.920532,
 'contour_right1_x': 54.373954000000005,
 'contour_right1_y': 4.8283269999999945,
 'contour_right2_x': 54.097148,
 'contour_right2_y': 10.930608,
 'contour_right3_x': 53.810076,
 'contour_right3_y': 17.246767999999996,
 'contour_right4_x': 53.074334,
 'contour_right4_y': 22.748478999999996,
 'contour_r

In [62]:
## 수집할 image url 
image_data = { '최규민:1':'https://fbcdn-sphotos-g-a.akamaihd.net/hphotos-ak-xta1/v/t1.0-9/12122398_906503296086905_9053689686321602818_n.jpg?oh=80fcaf06feebc607119139261daa82ee&oe=5696C59E&__gda__=1452163095_656cdb05f7f03f0920879b17b79041be'
              , '최규민:2' : 'https://fbcdn-sphotos-e-a.akamaihd.net/hphotos-ak-xpa1/t31.0-8/1517851_906503269420241_4763040186730610966_o.jpg'
              , '최규민:3' : 'https://fbcdn-sphotos-b-a.akamaihd.net/hphotos-ak-xtp1/v/t1.0-9/12118701_906503272753574_7177447444403669300_n.jpg?oh=3c594eb747cee0fad0417f661953779e&oe=56924967&__gda__=1456109393_d48c1b3f84d74dd8dbcaac4d31e12429'
              , '와이프:1' : 'https://fbcdn-sphotos-a-a.akamaihd.net/hphotos-ak-xpa1/v/t1.0-9/12108061_907705495966685_5481740761293586247_n.jpg?oh=398bdb849e0b91c4bb177ba84989302f&oe=56D21979&__gda__=1451609793_60d987ad6bc86213787e0530858396e1'
              , '아기:1' : 'https://scontent.xx.fbcdn.net/hphotos-xpt1/v/t1.0-9/12074727_907707092633192_9020724559277292808_n.jpg?oh=5759280c32c56e99159d974a13f44bcb&oe=569D6AFA'
              , '전명훈:1' : 'https://fbcdn-sphotos-h-a.akamaihd.net/hphotos-ak-xap1/v/t1.0-9/12140604_972525119474378_7475998966124883876_n.jpg?oh=383f560d042e194e9152c5919286fab8&oe=56CE8322&__gda__=1456399102_e2ac965c1f3878d6a66c966ec888e45a'
              , '전명훈:2' : 'https://fbcdn-sphotos-f-a.akamaihd.net/hphotos-ak-xap1/v/t1.0-9/12122612_972525132807710_7394758924527322556_n.jpg?oh=e422202838f486fdc2bb27941abb46a0&oe=568728E9&__gda__=1453230645_58f4314b4560b0a89a48287f87e31ebc'
              , '양정길:1' : 'https://fbcdn-sphotos-d-a.akamaihd.net/hphotos-ak-xpl1/v/t1.0-9/12096634_420268864849369_4650861756174901140_n.jpg?oh=3d043deaa1d94371c2889987e90b31fe&oe=569E8384&__gda__=1452120840_77764404987b26282bcbed4d93470751'
              , '양정길:2' : 'https://fbcdn-sphotos-f-a.akamaihd.net/hphotos-ak-xap1/v/t1.0-9/12074518_420268868182702_5807506611890573511_n.jpg?oh=3c43df8daa5b2e6cc13d30bef1bb976f&oe=568D00F3&__gda__=1453294639_f29144c19bc0b56a19f29d74d9ba3c79'
              , '양정길:3' : 'https://fbcdn-sphotos-h-a.akamaihd.net/hphotos-ak-xpt1/v/t1.0-9/12107776_420268898182699_873741782931745747_n.jpg?oh=89833ac3449bbb44fbfa60c47f0bb0a1&oe=5684CA0C&__gda__=1456380231_e9088b09832600a1eade233900ccb068'
              , '황영식:1' : 'https://fbcdn-sphotos-a-a.akamaihd.net/hphotos-ak-xat1/v/t1.0-9/12115767_10207581235506487_5394052949180960313_n.jpg?oh=256b9c02fc50d55894ee98e3f3f73a7e&oe=56D15270&__gda__=1455864498_d4fe6a6ffeaa6091ba8438bf29b2dd59'
              , '황영식:2' : 'https://fbcdn-sphotos-f-a.akamaihd.net/hphotos-ak-xap1/v/t1.0-9/12106836_10207581237506537_3414078523763358182_n.jpg?oh=39eec64bbba785acf0733fe2473643ee&oe=5686DE18&__gda__=1452742477_5c1c545b1cf419e6aadabe6ad22e2c98'
              , '김상범:1' : 'https://fbcdn-sphotos-f-a.akamaihd.net/hphotos-ak-xpf1/t31.0-8/s960x960/10984169_10207069759769838_1912549834994702083_o.jpg'
              , '김상범:2' : 'https://scontent.xx.fbcdn.net/hphotos-xta1/t31.0-8/12087157_10207069762649910_7045972924543347353_o.jpg'
              , '원빈:1' : 'http://image.fmkorea.com/files/attach/new/20141213/486616/85599953/86802006/da08e25010e6001bf565cd5cca3ca359.jpg'
              , '원빈:2' : 'http://cfile25.uf.tistory.com/image/1907CA414F11DCF31C96F2'
              , '원빈:3' : 'http://cfile7.uf.tistory.com/image/145E12504DAF7CEE198138'
              , '김수현:1' : 'http://cfile10.uf.tistory.com/image/167718454EB74A8C07D3DA'
              , '김수현:2' : 'http://image.ytn.co.kr/general/jpg/2013/1218/201312181042081641_h.jpg'
              , '김수현:3' : 'http://news.hankyung.com/nas_photo/201205/2012050296441_2012050290011.jpg'
              , '황준식:1' : 'https://fbcdn-sphotos-a-a.akamaihd.net/hphotos-ak-xta1/t31.0-8/11222922_10203543305263569_4994581988459995049_o.jpg'
              , '황준식:2' : 'https://fbcdn-sphotos-g-a.akamaihd.net/hphotos-ak-xpa1/v/t1.0-9/12107004_10203543304983562_8088447782909339952_n.jpg?oh=520c38aeb11958a8783e320c060b0ce4&oe=56CF5895&__gda__=1455730051_853f63114e12c1e049289327ea7abefc'
              , '황준식:3' : 'https://scontent.xx.fbcdn.net/hphotos-xpt1/v/t1.0-9/11224860_10203543305023563_4757525501530795128_n.jpg?oh=3360b213492cff2e1d1828adc8c66da3&oe=56865786'
              , '유호준:1' : 'https://fbcdn-sphotos-d-a.akamaihd.net/hphotos-ak-xtp1/v/t1.0-9/12109275_982753038456591_4936690200286712646_n.jpg?oh=94c185299b1a0b76d7006cce84999fb2&oe=56967D3A&__gda__=1455866636_1dec545f639a062a15a54b5381ce1753'
              , '유호준:2' : 'https://fbcdn-sphotos-a-a.akamaihd.net/hphotos-ak-xpa1/v/t1.0-9/12096516_982753118456583_1836928229776287106_n.jpg?oh=58abd369fa6acd9e64e708adab048562&oe=5693DD10&__gda__=1452569768_d7825274834cf0000e39fe497683e692'
             }


In [63]:
landmark_dict = {}
for name, img_url in image_data.items(): 
    try:
        keypoint = extract_face_landmark(img_url)
        landmark_dict[name] = get_feature_vector(keypoint)
    except:
        print ('extract fail : {0}'.format(name))

In [64]:
import pandas as pd
## dict to dataframe
landmark_ds = pd.DataFrame.from_dict( landmark_dict, orient='index')

In [73]:
landmark_ds

,eye_width,lip_to_nose,mouth_width,face_width,face_height,lip_to_chin,eye_to_eye,nose_width,eyebrow_width
김상범:1,0.65332,0.18036,0.42371,0.27891,1,0.27125,0.46367,0.28989,0.07309
김상범:2,0.45774,0.17712,0.30499,0.17130,1,0.25579,0.31862,0.20503,0.07676
김수현:1,0.45107,0.20541,0.22673,0.17002,1,0.25671,0.31702,0.19088,0.06413
김수현:2,0.38241,0.19790,0.20191,0.15363,1,0.24290,0.27322,0.16854,0.04680
김수현:3,0.70781,0.18121,0.40702,0.28452,1,0.21859,0.50424,0.32800,0.05449
아기:1,1.39357,0.22142,0.59198,0.46626,1,0.19154,0.99459,0.56177,0.05920
양정길:1,0.61196,0.20647,0.34220,0.23459,1,0.18553,0.43714,0.27378,0.06665
양정길:2,0.94308,0.20946,0.45293,0.35598,1,0.21260,0.66480,0.40625,0.06520
양정길:3,0.80375,0.20500,0.41902,0.28739,1,0.17990,0.57416,0.35779,0.06836
와이프:1,0.91300,0.19950,0.53354,0.28301,1,0.23461,0.65048,0.42445,0.05942


# raw facial landmark

In [66]:
## write to csv file
landmark_ds.to_csv('/Users/goodvc/data/fastcampus/week2/resource/landmark.csv')

In [67]:
## load from csv file
landmark_ds = pd.DataFrame.from_csv('/Users/goodvc/data/fastcampus/week2/resource/landmark.csv')

In [72]:
landmark_dict_raw = {}
for name, img_url in image_data.items(): 
    try:
        keypoint = extract_face_landmark(img_url)
        landmark_dict_raw[name] = extract_face_feature_raw(keypoint)
    except:
        print ('extract fail : {0}'.format(name))

In [69]:
import pandas as pd
## dict to dataframe
landmark_ds = pd.DataFrame.from_dict( landmark_dict, orient='index')
landmark_ds.to_csv('/Users/goodvc/data/fastcampus/week2/resource/landmark.csv')

In [71]:
landmark_raw_ds.head()

,mouth_lower_lip_bottom_y,mouth_lower_lip_left_contour2_y,nose_contour_right2_x,mouth_upper_lip_right_contour3_x,left_eyebrow_lower_left_quarter_y,contour_right2_y,contour_right4_y,mouth_lower_lip_right_contour3_x,nose_contour_left1_y,contour_right5_x,...,left_eye_upper_right_quarter_y,contour_left4_x,mouth_lower_lip_right_contour1_y,contour_left4_y,right_eyebrow_upper_right_quarter_y,contour_right1_y,left_eye_center_y,left_eye_left_corner_x,right_eye_upper_left_quarter_y,mouth_lower_lip_right_contour3_y
김상범:1,15.643000,14.406167,24.708547,24.551724,-5.387833,0.418834,8.599500,23.779812,-2.010833,34.510846,...,-3.037166,1.302069,13.293834,12.481500,-9.396000,-3.006000,-2.090166,8.145956,-3.924333,15.120334
김상범:2,22.236361,20.027231,17.981333,18.315833,-6.341773,3.904251,14.499388,17.332333,-0.818196,28.843667,...,-2.259075,0.849167,19.218572,15.420486,-9.465475,-1.111341,-1.318400,4.692167,-2.687498,21.304219
김수현:1,19.449750,17.910500,11.831666,10.592666,-2.186000,10.221500,16.506250,10.049000,3.639500,15.855666,...,2.017750,-0.434167,18.088250,10.849500,1.266000,6.801750,2.156500,4.028833,4.336250,19.796500
김수현:2,27.639778,26.222741,19.857709,20.445417,-6.596888,0.655556,13.403482,20.008542,-1.980740,28.997709,...,-3.382296,1.996667,23.070889,18.759778,-12.642814,-5.547407,-1.804518,5.703959,-4.995296,26.445704
김수현:3,39.166338,34.741338,41.073480,37.524680,-3.222148,15.240985,27.617746,35.514680,6.217394,52.405880,...,3.672323,0.004440,35.365633,21.055774,-0.711022,9.706831,4.071619,14.554680,6.426901,38.807887
